## ZADANIE 1 - Zaszumianie obrazów i NMSE
- Wczytujemy obraz w skali szarości (skimage.data.camera).
- Generujemy szumy: sól i pieprz (5%, 10%, 20%), Gaussa (sigma=0.05, 0.08, 0.1), jednostajny (h=+/-10, +/-20, +/-40).
- Dla każdego zaszumienia obliczamy NMSE względem oryginału.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.filters as filters
from skimage import data, util
from scipy.ndimage import median_filter
from skimage.util import random_noise
from skimage.filters import rank
from skimage.morphology import disk, square

In [ ]:
def calculate_nmse(original_img, processed_img):
    """
    Oblicza znormalizowany błąd średniokwadratowy (NMSE) między dwoma obrazami.
    Wzór: NMSE = suma((oryginał - zaszumiony)^2) / suma(oryginał^2)
    """
    # Konwersja obrazów do float64 dla precyzji obliczeń
    f = original_img.astype(np.float64)
    fe = processed_img.astype(np.float64)

    # Obliczenie licznika (suma kwadratów różnic)
    numerator = np.sum(np.square(f - fe))

    # Obliczenie mianownika (suma kwadratów pikseli oryginału)
    denominator = np.sum(np.square(f))

    # Unikanie dzielenia przez zero
    if denominator == 0:
        return 0.0

    return numerator / denominator

## Funkcje generujące szum

In [ ]:
def add_salt_and_pepper_noise(image, amount_percent):
    """Dodaje szum typu "sól i pieprz" do obrazu."""
    # random_noise działa na obrazach float w zakresie [0, 1]
    noisy_float = random_noise(image, mode='s&p', amount=amount_percent / 100.0)
    # Konwersja z powrotem do uint8 [0, 255]
    return util.img_as_ubyte(noisy_float)

def add_gaussian_noise(image, sigma):
    """Dodaje szum Gaussa do obrazu."""
    # Parametr 'var' to sigma^2
    noisy_float = random_noise(image, mode='gaussian', var=sigma**2)
    return util.img_as_ubyte(noisy_float)

def add_uniform_noise(image, h):
    """Dodaje szum jednostajny z przedziału [-h, +h] do obrazu."""
    # Stworzenie maski szumu o wymiarach obrazu
    noise_mask = np.random.uniform(-h, h, image.shape)

    # Dodanie maski do obrazu
    noisy_image = image.astype(np.float64) + noise_mask

    # Przycięcie wartości do zakresu [0, 255] i konwersja do uint8
    noisy_image_clipped = np.clip(noisy_image, 0, 255)

    return noisy_image_clipped.astype(np.uint8)

In [ ]:
# 1. Wczytanie obrazu w skali szarości
original_image = data.camera()

# 2. Zdefiniowanie parametrów szumu
sp_levels = [5, 10, 20]
gauss_sigmas = [0.05, 0.08, 0.1]
uniform_h_ranges = [10, 20, 40]

results = []

# 3. Zastosowanie szumu "sól i pieprz" i obliczenie NMSE
for p in sp_levels:
    noisy_img = add_salt_and_pepper_noise(original_image, p)
    nmse_val = calculate_nmse(original_image, noisy_img)
    results.append(('Sól i pieprz', f'{p}%', nmse_val, noisy_img))

# 4. Zastosowanie szumu Gaussa i obliczenie NMSE
for s in gauss_sigmas:
    noisy_img = add_gaussian_noise(original_image, s)
    nmse_val = calculate_nmse(original_image, noisy_img)
    results.append(('Gaussa', f's={s}', nmse_val, noisy_img))

# 5. Zastosowanie szumu jednostajnego i obliczenie NMSE
for h in uniform_h_ranges:
    noisy_img = add_uniform_noise(original_image, h)
    nmse_val = calculate_nmse(original_image, noisy_img)
    results.append(('Jednostajny', f'+/-{h}', nmse_val, noisy_img))

# 6. Wyświetlenie wyników w formie tabeli
print("--- Wyniki Zaszumiania i NMSE ---")
print(f"{'Rodzaj szumu':<15} | {'Parametr':<10} | {'NMSE':<20}")
print("-" * 50)
for name, param, nmse, _ in results:
    print(f"{name:<15} | {param:<10} | {nmse:.6f}")

# 7. Wizualizacja obrazu oryginalnego i przykładów szumów
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
ax = axes.ravel()

ax[0].imshow(original_image, cmap='gray')
ax[0].set_title("Oryginalny obraz")
ax[0].axis('off')

ax[1].imshow(results[0][3], cmap='gray')
ax[1].set_title(f"{results[0][0]} ({results[0][1]})")
ax[1].axis('off')

ax[2].imshow(results[3][3], cmap='gray')
ax[2].set_title(f"{results[3][0]} ({results[3][1]})")
ax[2].axis('off')

ax[3].imshow(results[6][3], cmap='gray')
ax[3].set_title(f"{results[6][0]} ({results[6][1]})")
ax[3].axis('off')

plt.tight_layout()
plt.show()

## ZADANIE 2 - Usuwanie szumu i wybór najlepszego filtru
Dla każdego zaszumionego obrazu testujemy filtry: medianowy (disk(1), disk(2), square(3)),
rank.mean (disk(1), disk(2), square(3)) oraz Gaussa (sigma in {0.5, 0.8, 1.0}).
Wybieramy filtr o najmniejszym NMSE(oryg, odszum).

In [ ]:

# Definicja siatki filtrów do przetestowania
filter_grid = [
    ('medianowy', {'fp': disk(1)}, "disk(1)"),
    ('medianowy', {'fp': disk(2)}, "disk(2)"),
    ('medianowy', {'fp': square(3)}, "square(3)"),
    ('uśredniający', {'fp': disk(1)}, "disk(1)"),
    ('uśredniający', {'fp': disk(2)}, "disk(2)"),
    ('uśredniający', {'fp': square(3)}, "square(3)"),
    ('Gaussa', {'sigma': 0.5}, "sigma=0.5"),
    ('Gaussa', {'sigma': 0.8}, "sigma=0.8"),
    ('Gaussa', {'sigma': 1.0}, "sigma=1.0"),
]

summary = []

# Iteracja po każdym zaszumionym obrazie z zadania 1
for kind, param_noisy_str, nmse_noisy, noisy_img in results:
    best_nmse = float('inf')
    best_filter_name = None
    best_filter_params_str = None

    # Testowanie każdego filtra na danym zaszumionym obrazie
    for name, params_dict, params_str in filter_grid:
        if name == 'medianowy':
            denoised_img = filters.median(noisy_img, footprint=params_dict['fp'])
        elif name == 'uśredniający':
            denoised_img = rank.mean(noisy_img, footprint=params_dict['fp'])
        else:  # 'Gaussa'
            denoised_float = filters.gaussian(noisy_img, sigma=params_dict['sigma'])
            denoised_img = util.img_as_ubyte(denoised_float)

        current_nmse = calculate_nmse(original_image, denoised_img)

        if current_nmse < best_nmse:
            best_nmse = current_nmse
            best_filter_name = name
            best_filter_params_str = params_str

    summary.append((kind, param_noisy_str, nmse_noisy, best_nmse, best_filter_name, best_filter_params_str))

# Wyświetlenie tabeli z wynikami zadania 2
print("\n--- ZADANIE 2: Podsumowanie Odszumiania ---")
header = f"{'Rodzaj szumu':<15} | {'Parametr':<10} | {'NMSE (zaszum.)':<15} | {'NMSE (odszum.)':<15} | {'Najlepszy filtr':<15} | {'Parametry'}"
print(header)
print("-" * len(header))
for kind, param, n_noisy, n_best, fname, fpar in summary:
    print(f"{kind:<15} | {param:<10} | {n_noisy:<15.6f} | {n_best:<15.6f} | {fname:<15} | {fpar}")

# Wizualizacja przykładu: oryginał, zaszumiony i najlepszy odszumiony
if summary:
    # Wybór pierwszego przypadku do wizualizacji
    kind, param_noisy_str, _, _, best_filter_name, best_filter_params_str = summary[0]
    noisy_img_to_show = results[0][3]

    # Ponowne zastosowanie najlepszego filtra
    best_filter_info = next(f for f in filter_grid if f[0] == best_filter_name and f[2] == best_filter_params_str)
    name, params_dict, _ = best_filter_info

    if name == 'medianowy':
        denoised_to_show = filters.median(noisy_img_to_show, footprint=params_dict['fp'])
    elif name == 'uśredniający':
        denoised_to_show = rank.mean(noisy_img_to_show, footprint=params_dict['fp'])
    else: # 'Gaussa'
        denoised_float = filters.gaussian(noisy_img_to_show, sigma=params_dict['sigma'])
        denoised_to_show = util.img_as_ubyte(denoised_float)

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(original_image, cmap='gray')
    axes[0].set_title("Oryginalny obraz")
    axes[0].axis('off')

    axes[1].imshow(noisy_img_to_show, cmap='gray')
    axes[1].set_title(f"Zaszumiony: {kind} ({param_noisy_str})")
    axes[1].axis('off')

    axes[2].imshow(denoised_to_show, cmap='gray')
    axes[2].set_title(f"Najlepsze odszumienie: {best_filter_name} ({best_filter_params_str})")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()


## # ZADANIE 3: Szum impulsowy dla obrazu barwnego

In [ ]:
def add_impulse_noise(image, p):

    noisy_image = image.copy()
    height, width, _ = noisy_image.shape

    # Obliczenie liczby pikseli do zmiany
    num_pixels_to_noise = int(height * width * p / 100)

    for _ in range(num_pixels_to_noise):
        # Losowanie współrzędnych piksela
        y = np.random.randint(0, height)
        x = np.random.randint(0, width)

        # Losowanie nowych wartości R, G, B
        R = np.random.randint(0, 256)
        G = np.random.randint(0, 256)
        B = np.random.randint(0, 256)

        # Ustawienie nowego koloru piksela
        noisy_image[y, x] = [R, G, B]

    return noisy_image

# Wczytanie obrazu barwnego
color_image_original = data.chelsea()

# Poziom zaszumienia
noise_percent = 3

# Dodanie szumu impulsowego
noisy_color_image = add_impulse_noise(color_image_original, noise_percent)

# Wizualizacja
print(f"\n--- ZADANIE 3: Szum impulsowy ({noise_percent}%) ---")
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(color_image_original)
axes[0].set_title("Oryginalny obraz barwny")
axes[0].axis('off')

axes[1].imshow(noisy_color_image)
axes[1].set_title(f"Obraz zaszumiony ({noise_percent}%)")
axes[1].axis('off')

plt.tight_layout()
plt.show()


# ZADANIE 4: NMSE dla obrazów kolorowych

In [ ]:
nmse_color_val = calculate_nmse(color_image_original, noisy_color_image)

print(f"Obliczona wartość NMSE dla obrazu barwnego (szum {noise_percent}%): {nmse_color_val:.6f}")

## ZADANIE 5: Implementacja Wektorowego Filtru Medianowego (VMF)

In [ ]:
def manhattan_distance(p1, p2):
    """Oblicza odległość w metryce miejskiej między dwoma wektorami kolorów."""
    return np.sum(np.abs(p1.astype(np.float64) - p2.astype(np.float64)))

def vector_median_filter(image):
    """
    Implementuje wektorowy filtr medianowy (VMF) z maską krzyżową 3x3.
    """
    # Obraz z dopełnieniem krawędzi w celu obsługi pikseli brzegowych
    padded_image = np.pad(image, ((1, 1), (1, 1), (0, 0)), mode='edge')
    output_image = np.copy(image)
    height, width, _ = image.shape

    # Iteracja przez każdy piksel oryginalnego obrazu
    for y in range(height):
        for x in range(width):
            # Współrzędne w obrazie z dopełnieniem
            py, px = y + 1, x + 1

            # Pobranie pikseli z maski krzyżowej
            p0 = padded_image[py, px]      # Centralny
            p1 = padded_image[py - 1, px]  # Górny
            p2 = padded_image[py + 1, px]  # Dolny
            p3 = padded_image[py, px - 1]  # Lewy
            p4 = padded_image[py, px + 1]  # Prawy

            mask_pixels = [p0, p1, p2, p3, p4]

            min_sum_dist = float('inf')
            vector_median = p0

            # Dla każdego piksela w masce, oblicz sumę jego odległości od pozostałych
            for pi in mask_pixels:
                current_sum_dist = 0
                for pj in mask_pixels:
                    current_sum_dist += manhattan_distance(pi, pj)

                # Jeśli znaleziona suma jest mniejsza, aktualizuj medianę wektorową
                if current_sum_dist < min_sum_dist:
                    min_sum_dist = current_sum_dist
                    vector_median = pi

            output_image[y, x] = vector_median

    return output_image

## Porównanie filtrów

In [ ]:
# Wczytanie obrazu
color_image_original = data.chelsea()

# Poziomy szumu do przetestowania
noise_levels = [2, 5, 10]
results_summary = []

print("--- ZADANIE 5: Porównanie Filtru Medianowego i VMF ---")
print("Obliczenia w toku...")

# Pętla po różnych poziomach szumu
for p in noise_levels:
    # 1. Zaszumianie obrazu
    noisy_image = add_impulse_noise(color_image_original, p)
    nmse_noisy = calculate_nmse(color_image_original, noisy_image)

    # 2. Odszumianie standardowym filtrem medianowym
    # Filtr median_filter z scipy działa na każdym kanale osobno
    denoised_standard_median = median_filter(noisy_image, size=(3, 3, 1))
    nmse_standard_median = calculate_nmse(color_image_original, denoised_standard_median)

    # 3. Odszumianie zaimplementowanym VMF
    denoised_vmf = vector_median_filter(noisy_image)
    nmse_vmf = calculate_nmse(color_image_original, denoised_vmf)

    results_summary.append((p, nmse_noisy, nmse_standard_median, nmse_vmf))

header = f"{'Wielkość szumu':<18} | {'NMSE (zaszum.)':<18} | {'NMSE (mediana std.)':<22} | {'NMSE (VMF)'}"
print("\n" + header)
print("-" * len(header))
for p, nmse_n, nmse_sm, nmse_v in results_summary:
    print(f"{f'impulsowy {p}%':<18} | {nmse_n:<18.6f} | {nmse_sm:<22.6f} | {nmse_v:.6f}")

# Wizualizacja dla ostatniego poziomu szumu (np. 10%)
last_noise_level = noise_levels[-1]
final_noisy_image = add_impulse_noise(color_image_original, last_noise_level)
final_denoised_median = median_filter(final_noisy_image, size=(3, 3, 1))
final_denoised_vmf = vector_median_filter(final_noisy_image)

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].imshow(color_image_original)
axes[0].set_title("Oryginał")
axes[0].axis('off')

axes[1].imshow(final_noisy_image)
axes[1].set_title(f"Szum impulsowy {last_noise_level}%")
axes[1].axis('off')

axes[2].imshow(final_denoised_median)
axes[2].set_title("Odszum. (mediana std.)")
axes[2].axis('off')

axes[3].imshow(final_denoised_vmf)
axes[3].set_title("Odszum. (VMF)")
axes[3].axis('off')

plt.tight_layout()
plt.show()